In [6]:
# | default_exp create_soil_parameters

In [7]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [9]:
# | export
from typing import Dict
from pathlib import Path
import collections
import os
import warnings
import numpy as np
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.compute_theta_at_given_p_soil import compute_theta_at_given_p_soil
from sureau_ecos_py.read_soil_file import read_soil_file

In [54]:
# | export
def create_soil_parameters(
    file_path: Path,  # Path to a csv file containing parameter values i.e path/to/parameter_values.csv
    modeling_options: Dict = None,  # Dictionary created using the `create_modeling_options` function
    list_of_parameters:List = None,  # A list containing the necessary input parameters instead of reading them in file. Will only be used if 'file_path' arguement is not provided
    default_soil: bool = False,  # A logical value indicating whether a default soil should be used  to run tests
    offset_psoil: int = 0,  # A numerical value indicating the offset in soil water potential (MPa)
    psoil_at_field_capacity:int = 33,
) -> Dict:

    "Create a Dictionary with soil parameters to run SureauR"

    # Create empty dictionary
    soil_params = collections.defaultdict(list)

    # file_path -----------------------------------------------------------------

    # Make sure the file_path exist or is None
    assert(
        file_path is None or os.path.exists(file_path)
    ), f'Path: {file_path} not found, check spelling or set file_path = None'


    # offset_psoil --------------------------------------------------------------
    assert (
        offset_psoil >= 0
    ), "offset_psoil must be an integer greater than or equal to 0"

    print(f"There is an offset on Psoil of {offset_psoil} MPa")
    soil_params["offset_psoil"] = offset_psoil

    # psoil_at_field_capacity ---------------------------------------------------
    assert (
        100 >= psoil_at_field_capacity >= 0
    ), "psoil_at_field_capacity must be an integer in the range between 0 and 100"

    print(f"Psoil at field capacity = {psoil_at_field_capacity/1000} MPa")
    soil_params["psoil_at_field_capacity"] = psoil_at_field_capacity / 1000

    # default soil for tests ----------------------------------------------------
    if default_soil is True:
        warnings.warn("Default soil used (Van-Genuchten Formulation)")

        soil_params["pedo_transfer_formulation"] = "vg"
        soil_params["rock_fragment_content"] = np.array([40, 75, 90])
        soil_params["depth"] = np.array([0.3, 1, 4], dtype=float)

        soil_params["layer_thickness"] = np.array([0, 0, 0], dtype=float)

        # Get the values of each soil depth
        soil_params["layer_thickness"][0] = soil_params["depth"][0]
        soil_params["layer_thickness"][1] = (

            soil_params["depth"][1] - soil_params["depth"][0]
        )
        soil_params["layer_thickness"][2] = (
            soil_params["depth"][2] - soil_params["depth"][1]
        )
        soil_params["g_soil0"] = 30

        # Van Genuchten parameters

        # Shape parameters of the relationship between soil water content and
        # soil water potential
        soil_params["alpha_vg"] = np.repeat(0.0035, 3)

        # Shape parameters of the relationship betwen soil water content and
        # soil water potential
        soil_params["n_vg"] = np.repeat(1.55, 3)

        # m parameters Van Genuchten equations
        soil_params["m"] = 1 - (1 / soil_params["n_vg"])

        # Shape parameters of the relationship between soil water content and
        # soil water potential
        soil_params["i_vg"] = np.repeat(0.5, 3)

        # Soil conductivity at saturation (mol/m/s/Mpa)
        soil_params["ksat_vg"] = np.repeat(1.69, 3)

        # Fraction of water at saturation capacity (cm3/cm3)
        soil_params["saturation_capacity_vg"] = np.repeat(0.5, 3)

        # Fraction of residual water (cm3/cm3)
        soil_params["residual_capacity_vg"] = np.repeat(0.1, 3)

        # add computation of wilting
        soil_params["wilting_point"] = compute_theta_at_given_p_soil(
            psi_target=1.5,
            theta_res=soil_params["residual_capacity_vg"],
            theta_sat=soil_params["saturation_capacity_vg"],
            alpha_vg=soil_params["alpha_vg"],
            n_vg=soil_params["n_vg"],
        )
        # add computation of field capacity from functions
        soil_params["field_capacity"] = compute_theta_at_given_p_soil(
            psi_target=psoil_at_field_capacity,
            theta_res=soil_params["residual_capacity_vg"],
            theta_sat=soil_params["saturation_capacity_vg"],
            alpha_vg=soil_params["alpha_vg"],
            n_vg=soil_params["n_vg"],
        )
        # Soil offset_psoil
        soil_params["offset_psoil"] = offset_psoil

    if default_soil is False and modeling_options is None:
        warnings.warn("modeling_options' is missing. Van Genuchten used as default")
        soil_params["pedo_transfer_formulation"] = "vg"

    if default_soil is False and modeling_options is not None:
        print(
            f'You are using {modeling_options["pedo_transfer_formulation"]} pedotransfer formulation'
        )
        soil_params["pedo_transfer_formulation"] = modeling_options["pedo_transfer_formulation"]


    # Read soil file ------------------------------------------------------------
    if file_path is not None:
       soil_params_file = read_soil_file(file_path, modeling_options)

    # Read list_of_parameters  --------------------------------------------------
    if file_path is None and list_of_parameters is not None:
        soil_params_file = list_of_parameters

    # Raise error if file_path and list_of_parameters both are missing ----------
    if file_path is None and list_of_parameters is None:
        raise ValueError("Both file_path and list_of_parameters are missing, provide one of them"
        )

    # Raise error if file_path and list_of_parameters both are provided ---------
    if file_path is not None and list_of_parameters is not None:
        raise ValueError("Both file_path and list_of_parameters are provided, only one of these two arguments should be used"
        )


    return soil_params

## Create modeling options dictionary

In [44]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)



## Create Soil parameters dictionary

In [56]:
create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    list_of_parameters="a",
    default_soil=True,
    offset_psoil=1,
    psoil_at_field_capacity=33,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = 0.033 MPa


/tmp/ipykernel_60651/3945327640.py:42: UserWarning: Default soil used (Van-Genuchten Formulation)
  warnings.warn("Default soil used (Van-Genuchten Formulation)")


ValueError: Both file_path and list_of_parameters are provided, only one of these two arguments should be used